In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys

print("install finished")

install finished


In [3]:
indicators = ["Overall Score", "International Students Ratio", "International Faculty Ratio", 
            "Faculty Student Ratio", "Citations per Faculty", "Academic Reputation", "Employer Reputation"]
print(len(indicators))
print(indicators)

7
['Overall Score', 'International Students Ratio', 'International Faculty Ratio', 'Faculty Student Ratio', 'Citations per Faculty', 'Academic Reputation', 'Employer Reputation']


In [4]:
def get_indicator_scores(soup):
    items = soup.select(".td-wrap-in")
    school, scores = [], [[] for i in range(7)]

    if(len(items) % 8 != 0):
        return [], []

    item =[]
    for index in range(len(items)):
        
        if(index % 8 == 0):
            item.append(items[index].contents[0].contents[0])
        else:
            try:
                test = float(items[index].contents[0])
                item.append(items[index].contents[0])
            except:
                item.append("NULL")  # NULL score

        if(index % 8 == 7):
            # append row to list
            school.append(item[0])
            if(len(item) != 8):
                print(item)
            for i in range(len(scores)):
                scores[i].append(item[i+1])

            item =[]
            continue
    
    return school, scores
    

In [21]:
def get_rank(soup):
    rows = soup.select("._univ-rank.show-this-in-mobile-indi")
    ranks = []
    for row in rows:
        ranks.append(row.contents[0])
    return ranks

In [11]:
import pandas as pd

def write_df_csv(rank, school, scores):
    df = pd.DataFrame()

    if(len(rank) != len(school)):
        print("length error:", len(rank), len(school))
        return False

    df["Rank"] = rank
    df["University"] = school
    
    for index in range(len(indicators)):
        df[indicators[index]] = scores[index]
        
    df.to_csv("QS-2022-ranking.csv", mode='a', header=False, index=False, encoding="utf-8")
    return True

In [23]:
file = open("QS-2022-ranking.csv", 'w')
file.write("Rank,University,")
for index in range(len(indicators)):
    file.write(indicators[index] + ",")
file.write("\n")
file.close()
print("finish csv init")

option = Options()
option.add_argument("--disable-notifications")

chrome = webdriver.Chrome('./chromedriver', options=option)
url = "https://www.topuniversities.com/university-rankings/world-university-rankings/2022"
chrome.get(url)
time.sleep(2)

# click ranking indicator tag
chrome.find_element_by_link_text("Rankings indicators", ).send_keys(Keys.ENTER)
time.sleep(3)

print("current page:", 1)
soup = BeautifulSoup(chrome.page_source, 'html.parser')
school, scores = get_indicator_scores(soup)
ranks = get_rank(soup)
result = write_df_csv(ranks, school, scores)
if(result == False):
    print("END with ERROR")

current_page = 2
while(True):
    print("current page:", current_page)
    try:
        # click next page
        chrome.find_element_by_class_name("next").send_keys(Keys.ENTER)
    except:
        break
    
    time.sleep(3)
    soup = BeautifulSoup(chrome.page_source, 'html.parser')
    
    school, scores = get_indicator_scores(soup)
    ranks = get_rank(soup)

    # write to csv
    result = write_df_csv(ranks, school, scores)
    if(result == False):
        break
    current_page += 1

chrome.close()
print("- finish -")


finish csv init
current page: 2
current page: 2
current page: 3
current page: 4
current page: 5
current page: 6
current page: 7
current page: 8
current page: 9
current page: 10
current page: 11
current page: 12
current page: 13
current page: 14
current page: 15
current page: 16
current page: 17
current page: 18
current page: 19
current page: 20
current page: 21
current page: 22
current page: 23
current page: 24
current page: 25
current page: 26
current page: 27
current page: 28
current page: 29
current page: 30
current page: 31
current page: 32
current page: 33
current page: 34
current page: 35
current page: 36
current page: 37
current page: 38
current page: 39
current page: 40
current page: 41
current page: 42
current page: 43
current page: 44
current page: 45
current page: 46
current page: 47
current page: 48
current page: 49
current page: 50
current page: 51
current page: 52
current page: 53
current page: 54
current page: 55
current page: 56
current page: 57
current page: 58
current

In [20]:
print(school)

['Massachusetts Institute of Technology (MIT) ', 'University of Oxford', 'Stanford University', 'University of Cambridge', 'Harvard University', 'California Institute of Technology (Caltech)', 'Imperial College London', 'ETH Zurich - Swiss Federal Institute of Technology', 'UCL', 'University of Chicago']
